# Pytorch版プログラム

In [1]:
import torch
import pyro
# random_seedの固定？
pyro.set_rng_seed(101)

In [2]:
# 平均0, 分散1のガウス分布からサンプル
loc = 0.   # mean zero
scale = 1. # unit variance
# locとscaleをセット
normal = torch.distributions.Normal(loc, scale) # create a normal distribution object
# 値をセットしたガウス分布から実際にxをサンプルする
x = normal.rsample() # draw a sample from N(0,1)
print("sample", x)
print("log prob", normal.log_prob(x)) # score the sample from N(0,1)

sample tensor(-1.3905)
log prob tensor(-1.8857)


In [11]:
def weather():
    # ベルヌーイ分布からのサンプル(1か0を出力)
    # Bernoulli(0.3)より, cloudyになる確率が0.3, sunnyが0.7になる
    cloudy = torch.distributions.Bernoulli(0.3).sample()
    # 前の行でcloudyに代入された値が1だったら'cloudy', 0だったら'sunny'が代入される
    cloudy = 'cloudy' if cloudy.item() == 1.0 else 'sunny'
    mean_temp = {'cloudy': 55.0, 'sunny': 75.0}[cloudy]
    scale_temp = {'cloudy': 10.0, 'sunny': 15.0}[cloudy]
    temp = torch.distributions.Normal(mean_temp, scale_temp).rsample()
    return cloudy, temp.item()

for _ in range(3):
    print(weather())

('sunny', 58.44780731201172)
('sunny', 80.37315368652344)
('sunny', 101.10397338867188)


# Pyro版プログラム

In [9]:
# "my_sample"の部分は任意の文字列で良い（？）
x = pyro.sample("my_sample", pyro.distributions.Normal(loc, scale))
print(x)

tensor(0.6033)


In [10]:
def weather():
    cloudy = pyro.sample('cloudy', pyro.distributions.Bernoulli(0.3))
    cloudy = 'cloudy' if cloudy.item() == 1.0 else 'sunny'
    mean_temp = {'cloudy': 55.0, 'sunny': 75.0}[cloudy]
    scale_temp = {'cloudy': 10.0, 'sunny': 15.0}[cloudy]
    temp = pyro.sample('temp', pyro.distributions.Normal(mean_temp, scale_temp))
    return cloudy, temp.item()

for _ in range(3):
    print(weather())

('sunny', 83.63148498535156)
('sunny', 75.91869354248047)
('sunny', 122.68901062011719)


In [12]:
def ice_cream_sales():
    cloudy, temp = weather()
    expected_sales = 200. if cloudy == 'sunny' and temp > 80.0 else 50.
    ice_cream = pyro.sample('ice_cream', pyro.distributions.Normal(expected_sales, 10.0))
    return ice_cream

In [16]:
def geometric(p, t=None):
    if t is None:
        t = 0
    x = pyro.sample("x_{}".format(t), pyro.distributions.Bernoulli(p))
    if x.item() == 1:
        return 0
    else:
        return 1 + geometric(p, t + 1)

print(geometric(0.5))

0


In [18]:
def normal_product(loc, scale):
    z1 = pyro.sample("z1", pyro.distributions.Normal(loc, scale))
    z2 = pyro.sample("z2", pyro.distributions.Normal(loc, scale))
    y = z1 * z2
    return y

def make_normal_normal():
    mu_latent = pyro.sample("mu_latent", pyro.distributions.Normal(0, 1))
    fn = lambda scale: normal_product(mu_latent, scale)
    return fn

print(make_normal_normal()(1.))

tensor(3.1065)
